In [2]:
import pandas as pd
import string
import unicodedata
import sys
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import sklearn.model_selection as cv
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import tensorflow as tf
plt.style.use("fivethirtyeight")
%matplotlib inline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
smileyfaces = [':-)', ':)', ':D', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)']
sadfaces = ['>:[', ':-(', ':(', ':-c', ':c', ':-<', ':<', ':-[', ':[', ':{', '=(','=[', 'D:']
angryfaces = ['>:(', '(╯°□°)╯︵ ┻━┻']
cryingfaces = [":’-(", ":’("]
skepticalfaces = ['>:', '>:/', ':-/', '=/',':L', '=L', ':S', '>.<']
noexpressionfaces = [':|', ':-|', '(｀・ω・´)']
surprisedfaces = ['>:O', ':-O', ':O', ':-o', ':o', '8O', 'O_O', 'o-o', 'O_o', 'o_O', 'o_o', 'O-O']

In [4]:
def cleanText(wordSeries):
    def remove_punctuation(x):
        for char in string.punctuation:
            x = x.replace(char, ' ')
        return x
    for smile in smileyfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(smile, ' smileyface '))
    for sad in sadfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(sad,' sadface '))
    for angry in angryfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(angry, ' angryface '))
    for cry in cryingfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(cry, ' cryingface '))
    for skeptical in skepticalfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(skeptical, ' skepticalface '))
    for noexp in noexpressionfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(noexp, ' noexpressionfaces '))
    for surprised in surprisedfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(surprised, ' surprisedface '))
    wordSeries = wordSeries.apply(lambda x: x.replace('...', ' dotdotdot '))
    wordSeries = wordSeries.apply(lambda x: x.replace('!', ' exclamatory '))
    wordSeries = wordSeries.apply(lambda x: remove_punctuation(x))
    wordSeries = wordSeries.apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
    wordSeries = wordSeries.apply(lambda x: x.lower())
    #wordSeries = wordSeries.apply(lambda x: x.replace('http', ' '))
    wordSeries = wordSeries.apply(lambda x: ' '.join( [w for w in x.split() if len(w)>1] ))
    return wordSeries

In [5]:
def tokenize(documents):
    documents = cleanText(documents)
    docs = [word_tokenize(content) for content in documents]
    stopwords_=set(stopwords.words('english'))
    def filter_tokens(sent):
        return([w for w in sent if not w in stopwords_])
    docs=list(map(filter_tokens,docs))
    lemmatizer = WordNetLemmatizer()
    docs_lemma = [[lemmatizer.lemmatize(word) for word in words] for words in docs]
    return docs_lemma

In [6]:
def createTFIDF(data, contentCol):
    data['Tokens'] = tokenize(data[contentCol])
    data['Tokens'] = data['Tokens'].apply(lambda x: ' '.join(x))
    corpus = [row for row in data['Tokens']]
    tfidf = TfidfVectorizer()
    document_tfidf_matrix = tfidf.fit_transform(corpus).toarray()
    document_tfidf_matrix = document_tfidf_matrix[:, :, None]
    return tfidf, document_tfidf_matrix

In [7]:
#dataPN = pd.read_csv('text_polarity.csv', encoding = "ISO-8859-1", header=None)
dataS = pd.read_csv('text_emotion.csv')

In [8]:
dataS['Clean'] = cleanText(dataS['content'])

In [64]:
dataS.head()

,tweet_id,sentiment,author,content,Clean
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,tiffanylue know was listenin to bad habit earl...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,layin bed with headache ughhhh dotdotdot waiti...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,funeral ceremony dotdotdot gloomy friday dotdo...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends soon exclamatory
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,dannycastillo we want to trade with someone wh...


In [9]:
dataS['BetterClean'] = tokenize(dataS['Clean'])
dataS['BetterClean'] = dataS['BetterClean'].apply(lambda x: " ".join(x))

In [67]:
dataS

,tweet_id,sentiment,author,content,Clean,BetterClean
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,tiffanylue know was listenin to bad habit earl...,tiffanylue know listenin bad habit earlier sta...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,layin bed with headache ughhhh dotdotdot waiti...,layin bed headache ughhhh dotdotdot waitin cal...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,funeral ceremony dotdotdot gloomy friday dotdo...,funeral ceremony dotdotdot gloomy friday dotdo...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends soon exclamatory,want hang friend soon exclamatory
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,dannycastillo we want to trade with someone wh...,dannycastillo want trade someone houston ticke...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,re pinging ghostridah why didn you go to prom ...,pinging ghostridah go prom bc bf like friend
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",should be sleep but im not exclamatory thinkin...,sleep im exclamatory thinking old friend want ...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,hmmm http www djhero com is down,hmmm http www djhero com
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,charviray charlene my love miss you,charviray charlene love miss
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,kelcouch sorry at least it friday,kelcouch sorry least friday


In [10]:
data = dataS[['sentiment','BetterClean']]

In [11]:
tokenizer = Tokenizer(num_words = 10000, split = " ")
tokenizer.fit_on_texts(data['BetterClean'].values)

textVector = tokenizer.texts_to_sequences(data['BetterClean'].values)
textVector = pad_sequences(textVector)

In [13]:
model = Sequential()
model.add(Embedding(10000, 256, input_length=textVector.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2))
model.add(LSTM(256, return_sequences=False, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(13, activation="softmax"))

In [14]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 77, 256)           2560000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 77, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 77, 256)           525312    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 13)                3341      
Total params: 3,613,965
Trainable params: 3,613,965
Non-trainable params: 0
_________________________________________________________________


In [15]:
y = pd.get_dummies(data['sentiment']).values

In [16]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(textVector, y, test_size=0.5)

In [17]:
batch_size = 32
epochs = 8
model.fit(Xtrain, Ytrain, epochs=5, batch_size=32, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 20000 samples
Epoch 1/5
20000/20000 [==============================] - 1018s 51ms/sample - loss: 2.0332 - acc: 0.2955
Epoch 2/5
20000/20000 [==============================] - 1023s 51ms/sample - loss: 1.8444 - acc: 0.3812
Epoch 3/5
20000/20000 [==============================] - 1025s 51ms/sample - loss: 1.7003 - acc: 0.4399
Epoch 4/5
20000/20000 [==============================] - 1013s 51ms/sample - loss: 1.5581 - acc: 0.4891
Epoch 5/5
20000/20000 [==============================] - 1007s 50ms/sample - loss: 1.3979 - acc: 0.5484


In [18]:
preds = model.predict(Xtest)

In [19]:
from sklearn.metrics import accuracy_score

In [21]:
Ytest[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [20]:
preds[0]

array([1.7120372e-04, 4.9815705e-04, 8.5358118e-04, 4.0275161e-04,
       1.4158637e-04, 1.8266856e-04, 4.9179975e-02, 1.8484649e-04,
       3.9006866e-04, 2.3025502e-03, 2.8707633e-02, 1.7433617e-02,
       8.9955139e-01], dtype=float32)

In [20]:
nonprobPreds = (preds == preds.max(axis=1)[:,None]).astype(int)

In [21]:
accuracy_score(Ytest, nonprobPreds)

0.30875